In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as SGfilter
from IPython.display import clear_output, display
import datetime
import joblib
from tqdm import tqdm
import sys
sys.path.append("../")
import environment

import gym
import ctfsql
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN
from stable_baselines3 import PPO

In [ ]:
n_simulations = 10
n_episodes_training = 10**6
n_episodes_test = 10**2

In [ ]:
env = gym.make('SQLstruct-v1')

In [ ]:
dqn_model = DQN('MlpPolicy', env, verbose=1, batch_size = 64) 

In [ ]:
dqn_model.learn(total_timesteps=10**5)
dqn_model.save('ignore_simulb64_')

In [ ]:
import evaluate_models as ev

rewards, num_steps = ev.test_episodes(dqn_model, env, num_episodes=1000, max_steps = 100)
len(rewards), len(num_steps)

In [ ]:
plt.plot(num_steps)

In [ ]:
plt.plot(rewards, "o")

In [ ]:
dqn_models = [DQN('MlpPolicy', env, verbose=(i==-1), batch_size = 64) 
              for i in range(n_simulations)]

In [ ]:
for i in tqdm(range(n_simulations)):
    dqn_models[i].learn(total_timesteps=10**6)
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
    dqn_models[i].save('ignore_'+ str(i) + '_simulb64_'+timestamp)

In [ ]:
rewards = []
num_steps = []
for i in tqdm(range(n_simulations)):
    reward, num_step = ev.test_episodes(dqn_models[i], env, num_episodes=1000, max_steps = 50)
    
    num_steps.append(np.array(num_step))
    rewards.append(np.array(reward))

In [ ]:
for i in range(n_simulations):
    plt.figure()
    plt.plot(num_steps, "o")
    plt.title(str(i))
plt.show()

In [ ]:
plt.boxplot(num_steps.T, notch = True, showmeans = True, meanline = True)
plt.savefig("figures/sim2_b64_test1.png")